In [34]:
from langchain.llms import Bedrock
from langchain.llms.bedrock import Bedrock as BedrockLLM
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import boto3
import json
import pandas as pd

In [35]:
# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'  # Change to your preferred region
)

In [36]:
def invoke_mistral_7b(prompt, max_tokens=512, temperature=0.9, top_p=0.9):
    """
    Invokes Mistral 7B Instruct model on AWS Bedrock
    
    Args:
        prompt (str): Input prompt
        max_tokens (int): Maximum tokens to generate
        temperature (float): Creativity control (0-1)
        top_p (float): Nucleus sampling threshold
        
    Returns:
        str: Generated response
    """
    # Format the prompt in Mistral's instruction format
    formatted_prompt = f"<s>[INST] {prompt} [/INST]"
    
    body = {
        "prompt": formatted_prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p
    }
    
    try:
        response = bedrock_runtime.invoke_model(
            body=json.dumps(body),
            modelId="mistral.mistral-7b-instruct-v0:2",
            accept='application/json',
            contentType='application/json'
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['outputs'][0]['text']
    
    except Exception as e:
        print(f"Error invoking model: {e}")
        return None

In [37]:
def get_chatbot_response(client, model_name, messages, temperature=0.9, max_tokens=512):
    """
    Gets chatbot response from Mistral 7B on AWS Bedrock
    
    Args:
        client: boto3 bedrock-runtime client
        model_name: Model ID (e.g., "mistral.mistral-7b-instruct-v0:2")
        messages: List of message dicts with "role" and "content"
        temperature: Creativity control (0-1)
        max_tokens: Maximum tokens to generate
        
    Returns:
        str: Generated response
    """
    # Format conversation history for Mistral
    formatted_prompt = ""
    for message in messages:
        if message["role"] == "system":
            formatted_prompt += f"<<SYS>>\n{message['content']}\n<</SYS>>\n\n"
        elif message["role"] == "user":
            formatted_prompt += f"<s>[INST] {message['content']} [/INST]"
        elif message["role"] == "assistant":
            formatted_prompt += f" {message['content']} </s>"
    
    body = {
        "prompt": formatted_prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": 0.8  # Matching your example
    }
    
    try:
        response = client.invoke_model(
            body=json.dumps(body),
            modelId=model_name,
            accept='application/json',
            contentType='application/json'
        )
        
        response_body = json.loads(response['body'].read())
        return response_body['outputs'][0]['text']
    
    except Exception as e:
        print(f"Error invoking model: {e}")
        return None

In [38]:
from dotenv import load_dotenv
import os
import json
from copy import deepcopy
import boto3

load_dotenv()

True

## Guard Agent

In [39]:
class GuardAgent():
    def __init__(self):
        self.client = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.getenv("AWS_REGION", "us-east-1")
        )
        self.model_name = os.getenv("BEDROCK_MODEL_NAME", "mistral.mistral-7b-instruct-v0:2")

    def get_response(self, message):
        messages = deepcopy(message)

        system_prompt = """<<SYS>>
        You are an helpful AI assistant for a Plant-selling store which sells plants and plant-related products.
        Your task is to determine whether user is asking something relevant to the plant store or not.

        The user is allowed to ask:
        1. Ask questions about the plant store like location, working hours, Fertilizers, compost, plants and plant shop related question.
        2. Make an order.
        3. Ask about recommendations of what to buy.

        The user is not allowed to ask:
        1. Ask about anything other than the plant store.
        2. Ask questions about the staff
        3. Ask about the owner of the store.

        Your output MUST be in this exact JSON format (no other text, no code blocks):

        {
            "chain_of_thought": "Analyze which point this input relates to",
            "decision": "allowed" or "not allowed",
            "message": "Your response to the user. If 'not allowed', use exactly: 'Sorry, I can't help you with that. Can I help you with something else?'"
        }
        <</SYS>>"""


        # Prepare messages in Mistral format
        input_messages = [{"role": "system", "content": system_prompt}] + messages[-3:]

        # Get response from Mistral
        chatbot_output = get_chatbot_response(
            client=self.client,
            model_name=self.model_name,
            messages=input_messages,
            temperature=0.1  # Lower temperature for more deterministic decisions
        )

        # Clean and verify the output
        chatbot_output = self.clean_json_output(chatbot_output)
        output = self.postprocess(chatbot_output)

        return output

    def clean_json_output(self, output):
        """Ensure the output is valid JSON with all required fields"""
        try:
            # Remove any code blocks and whitespace
            output = output.strip().replace("```json", "").replace("```", "")
            
            # Parse JSON
            parsed = json.loads(output)
            
            # Validate required fields
            required_fields = ["chain_of_thought", "decision", "message"]
            if not all(field in parsed for field in required_fields):
                raise ValueError("Missing required fields in JSON output")
                
            # Validate decision values
            if parsed["decision"] not in ["allowed", "not allowed"]:
                raise ValueError("Invalid decision value")
                
            return parsed
            
        except (json.JSONDecodeError, ValueError) as e:
            return {
                "chain_of_thought": f"Invalid response: {str(e)}",
                "decision": "not allowed",
                "message": "Sorry, I can't help you with that. Can I help you with something else?"
            }

    def postprocess(self, output):
        """
        Postprocess the output from the chatbot to ensure it is in the desired format.
        """
        if not isinstance(output, dict):
            output = {
                "chain_of_thought": "Invalid response format",
                "decision": "not allowed",
                "message": "Sorry, I can't help you with that. Can I help you with something else?"
            }

        # Ensure message is never empty for 'not allowed' decisions
        if output.get("decision", "not allowed") == "not allowed":
            output["message"] = output.get("message") or "Sorry, I can't help you with that. Can I help you with something else?"

        dict_output = {
            "role": "assistant",
            "content": output.get("message", ""),
            "memory": {
                "agent": "guard_agent",
                "guard_decision": output.get("decision", "not allowed"),
                "chain_of_thought": output.get("chain_of_thought", "")
            }
        }
        return dict_output

## Classification Agent

In [40]:
class ClassificationAgent():
    def __init__(self):
        self.client = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.getenv("AWS_REGION", "us-east-1")
        )
        self.model_name = os.getenv("BEDROCK_MODEL_NAME", "mistral.mistral-7b-instruct-v0:2")
    
    def get_response(self, messages):
        messages = deepcopy(messages)

        system_prompt = """<<SYS>>
You are a helpful AI assistant working for a Plant Shop application.

Your main task is to decide which specialized agent should handle the user's message.  
There are two agents you can choose from:

1. **details_agent**:  
   This agent handles questions only about:
   - Plant shop location
   - Plant shop working hours
   - Plant shop history
   - Delivery locations
   - Product collection and inventory
   - Prices of products

2. **order_taking_agent**:  
   This agent manages conversations where the user wants to place an order or is completing a purchase.

You must output your answer in the following strict JSON format:

{
    "chain_of_thought": "Explain why you chose a specific agent based on the user's message",
    "decision": "details_agent | order_taking_agent",
    "message": ""
}

Important:  
- Think carefully about the user's input before choosing the agent.  
- Follow the JSON format exactly without adding any extra text outside of it.
<</SYS>>"""
        
        # Format messages for Mistral
        formatted_messages = [{"role": "system", "content": system_prompt}] + messages[-3:]

        chatbot_output = get_chatbot_response(
            client=self.client,
            model_name=self.model_name,
            messages=formatted_messages,
            temperature=0.3  # Lower temperature for more consistent routing
        )

        # Clean and verify JSON output
        chatbot_output = self.clean_json_output(chatbot_output)
        output = self.postprocess(chatbot_output)
        return output

    def clean_json_output(self, output):
        """Ensure the output is valid JSON"""
        try:
            # Remove any markdown code blocks
            output = output.replace("```json", "").replace("```", "").strip()
            return json.loads(output)
        except json.JSONDecodeError:
            # Fallback to details agent if parsing fails
            return {
                "chain_of_thought": "Failed to parse response - defaulting to details agent",
                "decision": "details_agent",
                "message": ""
            }

    def postprocess(self, output):
        """Convert to standard agent output format"""
        if not isinstance(output, dict):
            output = {
                "chain_of_thought": "Invalid response format",
                "decision": "details_agent",
                "message": ""
            }

        return {
            "role": "assistant",
            "content": output.get("message", ""),
            "memory": {
                "agent": "classification_agent",
                "classification_decision": output.get("decision", "details_agent")
            }
        }

## Order Taking Agent

In [41]:
import os
import json
import re
import boto3
from copy import deepcopy
from dotenv import load_dotenv
from fuzzywuzzy import fuzz, process

load_dotenv()


class OrderTakingAgent():
    def __init__(self):
        self.client = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.getenv("AWS_REGION", "us-east-1")
        )
        self.model_name = os.getenv("BEDROCK_MODEL_NAME", "mistral.mistral-7b-instruct-v0:2")

        

        # Complete product list in the specified format
        self.products = [
            {"name": "White Butterfly (Syngonium Podophyllum)", "price": 100},
            {"name": "Peace Lily", "price": 150},
            {"name": "Chlorophytum Spider Plant", "price": 100},
            {"name": "Money Plant Marble Prince", "price": 150},
            {"name": "Snake Plant (Sansevieria)", "price": 200},
            {"name": "Aglaonema Lipstick", "price": 300},
            {"name": "Jade Plant (Portulacaria afra)", "price": 200},
            {"name": "Rubber Tree (Ficus elastica)", "price": 300},
            {"name": "Krishna Tulsi Plant (Black)", "price": 50},
            {"name": "Lemon Grass", "price": 50},
            {"name": "Curry Leaves", "price": 50},
            {"name": "Rama Tulsi Plant", "price": 50},
            {"name": "Ajwain Leaves", "price": 100},
            {"name": "Mentha Arvensis (Japanese Mint)", "price": 100},
            {"name": "Black Turmeric Plant (Black Haldi)", "price": 300},
            {"name": "Bhuiamla", "price": 100},
            {"name": "Wild Asparagus", "price": 200},
            {"name": "Jasminum sambac", "price": 150},
            {"name": "Parijat Tree", "price": 300},
            {"name": "Rose", "price": 100},
            {"name": "Raat Rani", "price": 200},
            {"name": "Shevanti", "price": 100},
            {"name": "Marigold (Orange)", "price": 50},
            {"name": "Champa (White)", "price": 200},
            {"name": "Rajnigandha", "price": 100},
            {"name": "Fragrant Panama rose", "price": 300},
            {"name": "Pincushion Cactus", "price": 150},
            {"name": "Bunny Ear Cactus", "price": 200},
            {"name": "Echinopsis chamaecereus", "price": 250},
            {"name": "Golden Pipe Cactus", "price": 300},
            {"name": "Moon Cactus (Grafted)", "price": 300},
            {"name": "Graptoveria opalina", "price": 250},
            {"name": "Crassula tetragona", "price": 200},
            {"name": "Aloe Vera", "price": 100},
            {"name": "Euphorbia (Red)", "price": 300},
            {"name": "Vermicompost", "price": 10, "unit": "kg"},
            {"name": "Vermicompost Mixture", "price": 20, "unit": "kg"},
            {"name": "Dec-Neemo (Bio-fertilizer)", "price": 150, "unit": "ltr"},
            {"name": "Dec-Mori (Bio-fertilizer)", "price": 150, "unit": "ltr"},
            {"name": "Agni Shield", "price": 300}
        ]

        # Add product aliases and normalization
        self.product_aliases = self._create_product_aliases()
        self.product_names = [p["name"] for p in self.products]
    
    def _create_product_aliases(self):
        return {
            # White Butterfly (Syngonium Podophyllum)
            "white butterfly": "White Butterfly (Syngonium Podophyllum)",
            "syngonium": "White Butterfly (Syngonium Podophyllum)",
            "arrowhead plant": "White Butterfly (Syngonium Podophyllum)",
            
            # Peace Lily
            "spathiphyllum": "Peace Lily",
            "white sails": "Peace Lily",
            "cobra plant": "Peace Lily",
            
            # Chlorophytum Spider Plant
            "spider plant": "Chlorophytum Spider Plant",
            "airplane plant": "Chlorophytum Spider Plant",
            "ribbon plant": "Chlorophytum Spider Plant",
            
            # Money Plant Marble Prince
            "money plant": "Money Plant Marble Prince",
            "marble queen": "Money Plant Marble Prince",
            "pothos": "Money Plant Marble Prince",
            
            # Snake Plant (Sansevieria)
            "mother in law's tongue": "Snake Plant (Sansevieria)",
            "sansevieria": "Snake Plant (Sansevieria)",
            "viper's bowstring hemp": "Snake Plant (Sansevieria)",
            
            # Aglaonema Lipstick
            "chinese evergreen": "Aglaonema Lipstick",
            "red aglaonema": "Aglaonema Lipstick",
            "painted drop tongue": "Aglaonema Lipstick",
            
            # Jade Plant (Portulacaria afra)
            "jade tree": "Jade Plant (Portulacaria afra)",
            "elephant bush": "Jade Plant (Portulacaria afra)",
            "dwarf jade": "Jade Plant (Portulacaria afra)",
            
            # Rubber Tree (Ficus elastica)
            "rubber plant": "Rubber Tree (Ficus elastica)",
            "ficus": "Rubber Tree (Ficus elastica)",
            "india rubber tree": "Rubber Tree (Ficus elastica)",
            
            # Krishna Tulsi Plant (Black)
            "black tulsi": "Krishna Tulsi Plant (Black)",
            "shyama tulsi": "Krishna Tulsi Plant (Black)",
            "krishna holy basil": "Krishna Tulsi Plant (Black)",
            
            # Lemon Grass
            "lemongrass": "Lemon Grass",
            "citronella": "Lemon Grass",
            "cymbopogon": "Lemon Grass",
            
            # Curry Leaves
            "kadi patta": "Curry Leaves",
            "sweet neem": "Curry Leaves",
            "murraya": "Curry Leaves",
            
            # Rama Tulsi Plant
            "green tulsi": "Rama Tulsi Plant",
            "rama holy basil": "Rama Tulsi Plant",
            "light holy basil": "Rama Tulsi Plant",
            
            # Ajwain Leaves
            "carom leaves": "Ajwain Leaves",
            "ajowan": "Ajwain Leaves",
            "bishop's weed": "Ajwain Leaves",
            
            # Mentha Arvensis (Japanese Mint)
            "wild mint": "Mentha Arvensis (Japanese Mint)",
            "corn mint": "Mentha Arvensis (Japanese Mint)",
            "field mint": "Mentha Arvensis (Japanese Mint)",
            
            # Black Turmeric Plant (Black Haldi)
            "kali haldi": "Black Turmeric Plant (Black Haldi)",
            "curcuma caesia": "Black Turmeric Plant (Black Haldi)",
            "black zedoary": "Black Turmeric Plant (Black Haldi)",
            
            # Bhuiamla
            "bhumi amla": "Bhuiamla",
            "stonebreaker": "Bhuiamla",
            "chanca piedra": "Bhuiamla",
            
            # Wild Asparagus
            "shatavari": "Wild Asparagus",
            "asparagus racemosus": "Wild Asparagus",
            "indian asparagus": "Wild Asparagus",
            
            # Jasminum sambac
            "arabian jasmine": "Jasminum sambac",
            "mogra": "Jasminum sambac",
            "sampaguita": "Jasminum sambac",
            
            # Parijat Tree
            "night jasmine": "Parijat Tree",
            "harsingar": "Parijat Tree",
            "tree of sorrow": "Parijat Tree",
            
            # Rose
            "gulab": "Rose",
            "rosa": "Rose",
            "flower queen": "Rose",
            
            # Raat Rani
            "night blooming jasmine": "Raat Rani",
            "cestrum": "Raat Rani",
            "queen of night": "Raat Rani",
            
            # Shevanti
            "chrysanthemum": "Shevanti",
            "guldaudi": "Shevanti",
            "autumn flower": "Shevanti",
            
            # Marigold (Orange)
            "genda": "Marigold (Orange)",
            "tagetes": "Marigold (Orange)",
            "orange genda": "Marigold (Orange)",
            
            # Champa (White)
            "plumeria": "Champa (White)",
            "frangipani": "Champa (White)",
            "white champa": "Champa (White)",
            
            # Rajnigandha
            "tuberose": "Rajnigandha",
            "polianthes": "Rajnigandha",
            "night queen": "Rajnigandha",
            
            # Fragrant Panama rose
            "panama rose": "Fragrant Panama rose",
            "sweet rose": "Fragrant Panama rose",
            "rondeletia": "Fragrant Panama rose",
            
            # Pincushion Cactus
            "mammillaria": "Pincushion Cactus",
            "cactus ball": "Pincushion Cactus",
            "nipple cactus": "Pincushion Cactus",
            
            # Bunny Ear Cactus
            "polka dot cactus": "Bunny Ear Cactus",
            "angel wings": "Bunny Ear Cactus",
            "opuntia": "Bunny Ear Cactus",
            
            # Echinopsis chamaecereus
            "peanut cactus": "Echinopsis chamaecereus",
            "chamaecereus": "Echinopsis chamaecereus",
            "mini cactus": "Echinopsis chamaecereus",
            
            # Golden Pipe Cactus
            "golden rat tail": "Golden Pipe Cactus",
            "cleistocactus": "Golden Pipe Cactus",
            "golden cactus": "Golden Pipe Cactus",
            
            # Moon Cactus (Grafted)
            "hibotan": "Moon Cactus (Grafted)",
            "grafted cactus": "Moon Cactus (Grafted)",
            "color top": "Moon Cactus (Grafted)",
            
            # Graptoveria opalina
            "opalina": "Graptoveria opalina",
            "ghost plant": "Graptoveria opalina",
            "pastel succulent": "Graptoveria opalina",
            
            # Crassula tetragona
            "mini pine tree": "Crassula tetragona",
            "crassula": "Crassula tetragona",
            "tetragona": "Crassula tetragona",
            
            # Aloe Vera
            "aloe": "Aloe Vera",
            "burn plant": "Aloe Vera",
            "first aid plant": "Aloe Vera",
            
            # Euphorbia (Red)
            "crown of thorns": "Euphorbia (Red)",
            "red spurge": "Euphorbia (Red)",
            "christ plant": "Euphorbia (Red)",
            
            # Vermicompost
            "worm compost": "Vermicompost",
            "vermi compost": "Vermicompost",
            "worm castings": "Vermicompost",
            
            # Vermicompost Mixture
            "vermi mix": "Vermicompost Mixture",
            "worm mix": "Vermicompost Mixture",
            "organic compost": "Vermicompost Mixture",
            
            # Dec-Neemo (Bio-fertilizer)
            "neem fertilizer": "Dec-Neemo (Bio-fertilizer)",
            "neem cake": "Dec-Neemo (Bio-fertilizer)",
            "organic neem": "Dec-Neemo (Bio-fertilizer)",
            
            # Dec-Mori (Bio-fertilizer)
            "seaweed fertilizer": "Dec-Mori (Bio-fertilizer)",
            "mori organic": "Dec-Mori (Bio-fertilizer)",
            "bio seaweed": "Dec-Mori (Bio-fertilizer)",
            
            # Agni Shield
            "plant protector": "Agni Shield",
            "disease shield": "Agni Shield",
            "plant guard": "Agni Shield"
        }


    def _match_product_name(self, user_input):
        """Fuzzy match product names with fallback to aliases"""
        # Try direct alias match first
        lower_input = user_input.lower()
        for alias, canonical in self.product_aliases.items():
            if alias in lower_input:
                return canonical
        
        # Fuzzy match with product names
        matches = process.extract(
            user_input,
            self.product_names,
            scorer=fuzz.token_sort_ratio,
            limit=3
        )
        
        # Return best match above 80% threshold
        if matches and matches[0][1] > 80:
            return matches[0][0]
        return None


    def get_response(self, messages):
        state = self._get_last_order_state(messages)
        
        # Force order flow continuity
        if state.get("step_number", 1) > 1:
            return self._continue_order_flow(messages, state)
            
        # Detect product names even in casual phrases
        user_input = messages[-1]['content']
        product_match = self._match_product_name(user_input)
        
        if product_match:
            return self._initiate_order(product_match)
            
        # Explicitly ask for clarification
        return {
            "role": "assistant",
            "content": "Which plant would you like to order? Here are our products: [list]",
            "memory": {"agent": "order_taking_agent"}
        }
    
    def _continue_order_flow(self, messages, state):
        if state["step_number"] == 2:
            return self._process_quantity(messages[-1]['content'], state)
        if state["step_number"] == 3:
            return self._confirm_order(state)
        return self._error_response("Invalid order state")
    
    def _initiate_order(self, product_name):
        product = next(p for p in self.products if p["name"] == product_name)
        return {
            "role": "assistant",
            "content": f"Great choice! How many {product_name} would you like?",
            "memory": {
                "agent": "order_taking_agent",
                "step_number": 2,
                "current_item": product_name,
                "order": []
            }
        }

    
    def _process_quantity(self, user_input, state):
        try:
            match = re.search(r'\d+', user_input)
            quantity = int(match.group()) if match else 1
            product = next(p for p in self.products if p["name"] == state["current_item"])
            
            new_item = {
                "item": product["name"],
                "quantity": quantity,
                "price": quantity * product["price"]
            }
            updated_order = state["order"] + [new_item]
            
            return {
                "role": "assistant",
                "content": f"Added {quantity} {product['name']}. Anything else?",
                "memory": {
                    "agent": "order_taking_agent",
                    "step_number": 3,
                    "order": updated_order
                }
            }
        except Exception as e:
            return {
                "role": "assistant",
                "content": "Please enter a valid quantity (e.g., '2 plants')",
                "memory": state
            }
    
    def _confirm_order(self, state):
        total = sum(item["price"] for item in state["order"])
        order_summary = "\n".join(
            f"- {item['item']} x{item['quantity']}: ₹{item['price']}"
            for item in state["order"]
        )
        
        return {
            "role": "assistant",
            "content": f"Your order:\n{order_summary}\nTotal: ₹{total}\nConfirm? (yes/no)",
            "memory": {
                "agent": "order_taking_agent",
                "step_number": 4,
                "order": state["order"]
            }
        }
        
    def _process_order_flow(self, messages):
        messages = deepcopy(messages)
        product_list = "\n".join(f"{p['name']} - ₹{p['price']}" for p in self.products)
        
        system_prompt = f"""<<SYS>>
You are an order processing bot. Products:
{product_list}

Follow these steps strictly:
1. Identify product(s)
2. Confirm quantity
3. Review order
4. Final confirmation

Respond in JSON format:
{{
    "step_number": 1-4,
    "order": [{{"item": "EXACT_NAME", "quantity": N}}],
    "response": "Message to user"
}}
<</SYS>>"""

        input_messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": messages[-1]['content']}
        ]

        try:
            chatbot_output = get_chatbot_response(
                client=self.client,
                model_name=self.model_name,
                messages=input_messages,
                temperature=0.3
            )
            output = self._clean_output(chatbot_output)
            
            return {
                "role": "assistant",
                "content": output.get("response", "How can I assist with your order?"),
                "memory": {
                    "agent": "order_taking_agent",
                    "step_number": output.get("step_number", 1),
                    "order": output.get("order", [])
                }
            }
        except Exception as e:
            return self._error_response(str(e))

    def _get_last_order_state(self, messages):
        """Extract last order state from conversation history"""
        for message in reversed(messages):
            if message.get("role") == "assistant":
                memory = message.get("memory", {})
                if memory.get("agent") == "order_taking_agent":
                    return {
                        "step": memory.get("step_number", 1),
                        "order": memory.get("order", []),
                        "asked_recommendation": memory.get("asked_recommendation_before", False)
                    }
        return {"step": 1, "order": [], "asked_recommendation": False}

    def _clean_output(self, raw_output):
        """Ensure valid JSON output structure"""
        try:
            # Remove any code blocks
            clean_output = raw_output.replace("```json", "").replace("```", "").strip()
            output = json.loads(clean_output)
            
            # Validate required fields
            if not all(k in output for k in ["step_number", "order", "response"]):
                raise ValueError("Missing required fields")
                
            # Ensure order is a list
            if isinstance(output["order"], str):
                output["order"] = json.loads(output["order"])
                
            return output
            
        except Exception as e:
            print(f"Error cleaning output: {e}")
            return {
                "step_number": 1,
                "order": [],
                "response": "Sorry, I'm having trouble processing your order. Please try again."
            }



    def _error_response(self, error_msg):
        """Generate error response"""
        print(f"OrderTakingAgent error: {error_msg}")
        return {
            "role": "assistant",
            "content": "Sorry, I'm having trouble with your order. Please try again.",
            "memory": {
                "agent": "order_taking_agent",
                "step_number": 1,
                "order": [],
                "asked_recommendation_before": False
            }
        }

## Details Agent

In [42]:
import faiss
import numpy as np
from datetime import datetime 


class DetailsAgent():
    def __init__(self):
        # Initialize chat model client
        self.client = boto3.client(
            service_name='bedrock-runtime',
            region_name=os.getenv("AWS_REGION", "us-east-1")
        )
        self.chat_model_id = os.getenv("BEDROCK_MODEL_NAME", "mistral.mistral-7b-instruct-v0:2")
        
        # Load knowledge documents directly
        self.knowledge_base = self._load_knowledge_documents(
            documents={
                "about_us": r"A:\NLP Projects\Chatbot\python_code\Experiments\Plantify_about_us.txt",
                "price_list": r"A:\NLP Projects\Chatbot\python_code\Experiments\price_list_text.txt"
            }
        )

    def _load_knowledge_documents(self, documents):
        """Load and validate knowledge documents"""
        knowledge = {}
        try:
            for doc_name, doc_path in documents.items():
                if not os.path.exists(doc_path):
                    raise FileNotFoundError(f"Document not found: {doc_path}")
                
                with open(doc_path, 'r', encoding='utf-8') as f:
                    content = f.read().strip()
                
                if not content:
                    raise ValueError(f"Empty document: {doc_path}")
                
                knowledge[doc_name] = {
                    "content": content,
                    "path": doc_path,
                    "last_modified": os.path.getmtime(doc_path)
                }
            
            print("Loaded knowledge documents:")
            for doc_name, data in knowledge.items():
                print(f"- {doc_name}: {len(data['content'])} chars, last modified {datetime.fromtimestamp(data['last_modified'])}")
            
            return knowledge
        except Exception as e:
            print(f"Failed to load knowledge base: {e}")
            raise

    def _select_relevant_documents(self, user_message):
        """Determine which documents are relevant to the query"""
        query_lower = user_message.lower()
        relevant_docs = []
        
        # Simple keyword-based routing
        if any(word in query_lower for word in ["price", "cost", "how much"]):
            relevant_docs.append("price_list")
        
        if any(word in query_lower for word in ["store", "location", "hour", "deliver", "about"]):
            relevant_docs.append("about_us")
        
        # If no specific match, use all documents
        return relevant_docs if relevant_docs else list(self.knowledge_base.keys())

    def get_response(self, messages):
        user_message = messages[-1]['content']
        
        try:
            # Select relevant documents
            relevant_docs = self._select_relevant_documents(user_message)
            print(f"Selected documents: {relevant_docs}")
            
            # Prepare context with document boundaries
            context = []
            for doc_name in relevant_docs:
                doc_data = self.knowledge_base[doc_name]
                context.append(
                    f"===== {doc_name.upper().replace('_', ' ')} =====\n"
                    f"{doc_data['content']}\n"
                )
            
            prompt = f"""<<SYS>>
You are a precise Plantify store assistant. Use ONLY the following documents to answer.
If the information isn't present, respond: "This information isn't available in our records. Please visit www.plantify.com for details."

DOCUMENTS:
{"".join(context)}
<</SYS>>

Question: {user_message}

Answer concisely and accurately:"""
            
            # Get response with supported parameters only
            response = get_chatbot_response(
                client=self.client,
                model_name=self.chat_model_id,
                messages=[{
                    "role": "user",
                    "content": prompt
                }],
                temperature=0.1,
                max_tokens=300
                # Removed the unsupported top_p parameter
            )
            
            # Post-process response
            response = response.strip()
            if not response or any(phrase in response.lower() for phrase in ["i don't know", "not available"]):
                response = "This information isn't available in our records. Please visit www.plantify.com for details."
            
            return {
                "role": "assistant",
                "content": response,
                "memory": {
                    "agent": "details_agent",
                    "sources": relevant_docs,
                    "documents_used": len(relevant_docs)
                }
            }
            
        except Exception as e:
            print(f"Error in DetailsAgent: {e}")
            return {
                "role": "assistant",
                "content": "I'm experiencing technical difficulties. Please try again later or contact support.",
                "memory": {
                    "agent": "details_agent",
                    "error": str(e)
                }
            }
        
        
    def postprocess(self, output):
        """Maintain compatibility with existing code"""
        if isinstance(output, dict):
            return output
        return {
            "role": "assistant",
            "content": output,
            "memory": {"agent": "details_agent"}
        }

## Agent Testing

In [43]:
import os
from copy import deepcopy

def test_full_agent_flow():
    """Test the complete agent flow (Guard → Classification → Specific Agent)"""
    print("=== Full Agent Testing Mode ===")
    print("Type 'exit' to quit\n")
    
    # Initialize all agents
    guard = GuardAgent()
    classifier = ClassificationAgent()
    details_agent = DetailsAgent()
    order_taking_agent = OrderTakingAgent()
    
    conversation_history = []
    order_complete = False
    
    while True:
        if order_complete:
            print("\n[Order Complete] Thank you for ordering from Plantify! Your order has been confirmed.")
            break
            
        # Get user input
        user_input = input("You: ").strip().lower()
        
        if user_input in ('exit', 'quit'):
            print("\nExiting test mode...")
            break
            
        if not user_input:
            continue
            
        # Check for order completion phrases
        if any(phrase in user_input for phrase in ["no that's all", "no thanks", "nothing else", "that's it"]):
            order_complete = True
            continue
            
        # Add to conversation history
        conversation_history.append({
            "role": "user",
            "content": user_input
        })
        
        # 1. First, get guard agent's response
        guard_response = guard.get_response(conversation_history)
        
        # Display guard agent analysis
        print("\n[Guard Agent Analysis]")
        print(f"Decision: {guard_response['memory']['guard_decision']}")
        print(f"Response: {guard_response['content']}")
        if "chain_of_thought" in guard_response.get("memory", {}):
            print(f"Reasoning: {guard_response['memory']['chain_of_thought']}")
        
        # Only proceed if allowed by guard
        if guard_response['memory']['guard_decision'] == "allowed":
            # 2. Get classification agent's response
            classification_response = classifier.get_response(conversation_history)
            
            # Display classification results
            print("\n[Classification Agent Analysis]")
            print(f"Selected Agent: {classification_response['memory']['classification_decision']}")
            if "chain_of_thought" in classification_response.get("memory", {}):
                print(f"Reasoning: {classification_response['memory']['chain_of_thought']}")
            
            # 3. Route to the appropriate agent
            selected_agent = classification_response['memory']['classification_decision']
            agent_response = None
            
            if selected_agent == "details_agent":
                print("\n[Routing to Details Agent]")
                agent_response = details_agent.get_response(conversation_history)
                
            elif selected_agent == "order_taking_agent":
                print("\n[Routing to Order Taking Agent]")
                agent_response = order_taking_agent.get_response(conversation_history)
                
                # Check if this is a final confirmation step
                if agent_response['memory'].get('step_number', 1) >= 4:  # Assuming step 4 is final confirmation
                    order_complete = True
                    continue
                
                # Additional order validation and calculation
                if "order" in agent_response.get("memory", {}):
                    order = agent_response['memory']['order']
                    if order:
                        total = sum(item.get('price', 0) * item.get('quantity', 1) for item in order)
                        print(f"\n[Order Calculation]")
                        print("Current Order Items:")
                        for item in order:
                            print(f"- {item.get('item', 'Unknown')}: {item.get('quantity', 1)} x ₹{item.get('price', 0)}")
                        print(f"TOTAL: ₹{total}")
            
            else:
                agent_response = {
                    "role": "assistant",
                    "content": f"Agent '{selected_agent}' is not yet implemented",
                    "memory": {"agent": selected_agent}
                }
            
            # Display final agent response with proper error handling
            print("\n[Agent Response]")
            print(f"Response: {agent_response['content']}")
            
            # Safely handle memory fields
            memory = agent_response.get("memory", {})
            if "sources" in memory:
                print(f"Sources: {memory['sources']}")
            if "scores" in memory:
                print(f"Confidence Scores: {memory['scores']}")
            if "error" in memory:
                print(f"Error Details: {memory['error']}")
            if "recommended_items" in memory:
                print(f"Recommended Items: {memory['recommended_items']}")
            if "recommendation_type" in memory:
                print(f"Recommendation Type: {memory['recommendation_type']}")
            if "step_number" in memory:
                print(f"Order Step: {memory['step_number']}")
            
            # Add all responses to history
            conversation_history.extend([
                guard_response,
                classification_response,
                agent_response
            ])
        else:
            # Add only guard response to history if not allowed
            conversation_history.append(guard_response)
        
        print()  # Add spacing between turns

if __name__ == "__main__":
    test_full_agent_flow()

=== Full Agent Testing Mode ===
Type 'exit' to quit

Loaded knowledge documents:
- about_us: 2805 chars, last modified 2025-04-01 03:56:48.411321
- price_list: 1063 chars, last modified 2025-04-03 21:19:11.508162



[Guard Agent Analysis]
Decision: allowed
Response: Great! Which type of rose would you like to order? We have a variety of roses available.
Reasoning: The user is asking to make an order, which is allowed.

[Classification Agent Analysis]
Selected Agent: order_taking_agent

[Routing to Order Taking Agent]

[Agent Response]
Response: Which plant would you like to order? Here are our products: [list]


[Guard Agent Analysis]
Decision: allowed
Response: Great choice! Our rose plant is beautiful and fragrant. I'll connect you with our order_taking_agent to help you place your order. Do you have a preferred variety or size?
Reasoning: User is asking about making an order, specifically for a rose plant. This is allowed.

[Classification Agent Analysis]
Selected Agent: order_taking_agent

[Routing to Order Taking Agent]

[Agent Response]
Response: Which plant would you like to order? Here are our products: [list]


[Guard Agent Analysis]
Decision: allowed
Response: Great choice! Could you pl